In [1]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, Activation
from keras.models import Sequential
from keras.utils import to_categorical

In [3]:
data = pd.read_csv('./pokemon_aug.csv')
data = data.drop(['Total', 'HP', 'Attack', 'Defense','Sp. Atk','Sp. Def',
                  'Speed', 'Generation', 'Legendary'], axis='columns')

In [23]:
data.Type1.unique()

array(['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric',
       'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice',
       'Dragon', 'Dark', 'Steel', 'Flying'], dtype=object)

In [25]:
bug = os.listdir('images/scraped/bug/')

In [28]:
bug = pd.DataFrame(scraped, columns=['filename'])

In [31]:
bug['type'] = "Bug"

In [33]:
bug.tail()

,filename,type
59,bug1-30.jpg,Bug
60,bug1-24.jpg,Bug
61,bug1-19.jpg,Bug
62,bug1-25.jpg,Bug
63,bug1-31.png,Bug


In [34]:
grass = os.listdir('images/scraped/grass/')
fire = os.listdir('images/scraped/fire/')
water = os.listdir('images/scraped/water/')
normal = os.listdir('images/scraped/normal/')
poison = os.listdir('images/scraped/poison/')
ele = os.listdir('images/scraped/ele/')
ground = os.listdir('images/scraped/ground/')
fairy = os.listdir('images/scraped/fairy/')
fight = os.listdir('images/scraped/fight/')
psy = os.listdir('images/scraped/psy/')
rock = os.listdir('images/scraped/rock/')
ghost = os.listdir('images/scraped/ghost/')
ice = os.listdir('images/scraped/ice/')
drag = os.listdir('images/scraped/dragon/')
dark = os.listdir('images/scraped/dark/')
steel = os.listdir('images/scraped/steel/')
fly = os.listdir('images/scraped/fly/')

In [35]:
grass = pd.DataFrame(grass, columns=['filename'])
fire = pd.DataFrame(fire, columns=['filename'])
water = pd.DataFrame(water, columns=['filename'])
normal = pd.DataFrame(normal, columns=['filename'])
poison = pd.DataFrame(poison, columns=['filename'])
ele = pd.DataFrame(ele, columns=['filename'])
ground = pd.DataFrame(ground, columns=['filename'])
fairy = pd.DataFrame(fairy, columns=['filename'])
fight = pd.DataFrame(fight, columns=['filename'])
psy = pd.DataFrame(psy, columns=['filename'])
rock = pd.DataFrame(rock, columns=['filename'])
ghost = pd.DataFrame(ghost, columns=['filename'])
ice = pd.DataFrame(ice, columns=['filename'])
drag = pd.DataFrame(drag, columns=['filename'])
dark = pd.DataFrame(dark, columns=['filename'])
steel = pd.DataFrame(steel, columns=['filename'])
fly = pd.DataFrame(fly, columns=['filename'])

In [36]:
grass['Type1'] = "Grass"
fire['Type1'] = "Fire"
water['Type1'] = "Water"
normal['Type1'] = "Normal"
poison['Type1'] = "Poison"
ele['Type1'] = "Electric"
ground['Type1'] = "Ground"
fairy['Type1'] = "Fairy"
fight['Type1'] = "Fight"
psy['Type1'] = "Psychic"
rock['Type1'] = "Rock"
ghost['Type1'] = "Ghost"
ice['Type1'] = "Ice"
drag['Type1'] = "Dragon"
dark['Type1'] = "Dark"
steel['Type1'] = "Steel"
fly['Type1'] = "Flying"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 2 columns):
filename    79 non-null object
Type1       79 non-null object
dtypes: object(2)
memory usage: 1.3+ KB


----

In [ ]:
mask = np.random.rand(len(data)) < 0.8

In [ ]:
train = data[mask].reset_index()
test = data[~mask].reset_index()

In [ ]:
gen = ImageDataGenerator(vertical_flip=True, validation_split=0.2)

In [ ]:
traingen = gen.flow_from_dataframe(train, directory='./images/models/', x_col='fileid', y_col='Type1',
                                   class_mode='categorical', subset='training', batch_size=5)
validgen = gen.flow_from_dataframe(train, directory='./images/models/', x_col='fileid', y_col='Type1',
                                   class_mode='categorical', subset='validation', batch_size=5)

In [ ]:
testdatagen = ImageDataGenerator(vertical_flip=False)
testgen = testdatagen.flow_from_dataframe(test, directory='./images/models', x_col='fileid', y_col='Type1', batch_size=5)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=2, padding='same', activation='relu', 
                 input_shape=(256, 256, 3)))
model.add(Conv2D(32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=2, activation='relu'))
model.add(Conv2D(64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(18, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
STEP_SIZE_TRAIN=traingen.n//traingen.batch_size
STEP_SIZE_VALID=validgen.n//validgen.batch_size
step_size_test=testgen.n//testgen.batch_size

In [ ]:
model.fit_generator(generator=traingen,
                    epochs=10,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                   validation_data=validgen,
                   validation_steps=STEP_SIZE_VALID)

In [ ]:
testgen.reset()
pred = model.predict_generator(testgen, steps = step_size_test, verbose=1)

In [ ]:
pred[2]

In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)

In [ ]:
predicted_class_indices

In [ ]:
labels = (testgen.class_indices)
print(labels)

In [ ]:
labels = dict((v,k) for k,v in labels.items())
print(labels)

In [ ]:
predictions = [labels[k] for k in predicted_class_indices]
print(predictions[-1])

In [ ]:
len(predictions)

In [ ]:
len(testgen.filenames)

In [ ]:
filenames=testgen.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)

In [ ]:
results.groupby

In [ ]:
data.head()

In [ ]:
data.info()